### In this project, Pyspark DataFrames were created for the txt format of KJV Bible after being processed with RDD. Additional columns were made for SQL query purposes.

#### Load KJV txt file from external path

In [2]:
import re

In [3]:
k = sc.textFile("/Users/binggangliu/Downloads/Bible/kjv/KJV_c.txt", minPartitions=4)
k.count()

32423

In [4]:
k.take(5)

['',
 'Genesis',
 '',
 'Gen.1:1 In the beginning God created the heaven and the earth.',
 'Gen.1:2 And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.']

#### Prepare data for DataFrame, first remove the empty lines and the lines with book names, and only keep the lines for all verses

In [5]:
def find_verses(line):
    pattern = re.compile(r'\.(\d|\d\d|\d\d\d):')
    match = pattern.findall(line)
    if match:
        return line

In [6]:
kv = k.filter(lambda x: find_verses(x))

In [7]:
kv.take(5)

['Gen.1:1 In the beginning God created the heaven and the earth.',
 'Gen.1:2 And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.',
 'Gen.1:3 And God said, Let there be light: and there was light.',
 'Gen.1:4 And God saw the light, that it was good: and God divided the light from the darkness.',
 'Gen.1:5 And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.']

##### Total number of verses:

In [8]:
kv.count()

31102

#### Splitting the verses

In [9]:
def split(lines):
    lines = lines.split()
    return lines

kv_split = kv.map(split)

In [10]:
kv_split.take(2)

[['Gen.1:1',
  'In',
  'the',
  'beginning',
  'God',
  'created',
  'the',
  'heaven',
  'and',
  'the',
  'earth.'],
 ['Gen.1:2',
  'And',
  'the',
  'earth',
  'was',
  'without',
  'form,',
  'and',
  'void;',
  'and',
  'darkness',
  'was',
  'upon',
  'the',
  'face',
  'of',
  'the',
  'deep.',
  'And',
  'the',
  'Spirit',
  'of',
  'God',
  'moved',
  'upon',
  'the',
  'face',
  'of',
  'the',
  'waters.']]

#### Keep the verse title, and join the remaining words together as a single verse

Generate an index for each verse, as the verse ID (VID)

In [11]:
from pyspark.sql import Row

In [12]:
kv1 = kv_split.zipWithIndex()

In [13]:
kv1.take(2)

[(['Gen.1:1',
   'In',
   'the',
   'beginning',
   'God',
   'created',
   'the',
   'heaven',
   'and',
   'the',
   'earth.'],
  0),
 (['Gen.1:2',
   'And',
   'the',
   'earth',
   'was',
   'without',
   'form,',
   'and',
   'void;',
   'and',
   'darkness',
   'was',
   'upon',
   'the',
   'face',
   'of',
   'the',
   'deep.',
   'And',
   'the',
   'Spirit',
   'of',
   'God',
   'moved',
   'upon',
   'the',
   'face',
   'of',
   'the',
   'waters.'],
  1)]

Separate the title from the verse as verse name (VName)

In [14]:
ktv = kv1.map(lambda r: Row(VName = r[0][0], verse = ' '.join(r[0][1:]), VID= r[1]+1))

In [15]:
ktv.take(3)

[Row(VID=1, VName='Gen.1:1', verse='In the beginning God created the heaven and the earth.'),
 Row(VID=2, VName='Gen.1:2', verse='And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.'),
 Row(VID=3, VName='Gen.1:3', verse='And God said, Let there be light: and there was light.')]

#### Convert to DataFrame from RDD

In [16]:
df = ktv.toDF()

In [18]:
df.take(2)

[Row(VID=1, VName='Gen.1:1', verse='In the beginning God created the heaven and the earth.'),
 Row(VID=2, VName='Gen.1:2', verse='And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.')]

In [19]:
from pyspark.sql.functions import col, udf

#from pyspark.sql.functions import monotonically_increasing_id
#df0 = df.withColumn('VID', monotonically_increasing_id())
#Initially the monotonically_increasing_id() method was used to generate the unique ID's for the verses,
# but the results showed that the consecutiveness was not guaranteed by this method, hence the zipWithIndex() method for RDD is used above


#### Add columns for book name (BName) and chapter name (CName), generating the names by extracting the required characters from the 'VName' column

In [20]:

bookname_udf = udf(lambda VName: VName[:2] if VName[2]=='.' else VName[:3] if VName[3]=='.' else VName[:4] if VName[4]=='.' else VName[:5])
chaptername_udf = udf(lambda VName: VName[:4] if VName[4]==':' else VName[:5] if VName[5]==':' else VName[:6] if VName[6]==':' else VName[:7] if VName[7]==':' else VName[:8])


df1 = df.withColumn('BName', bookname_udf(df.VName)).withColumn('CName', chaptername_udf(df.VName))

#### Create a function below to generate ID's for book name (BName) and chapeter name (CName)

Because rows in Pyspark a DataFrame are not iterable, the function below cannot take 'CName' and 'BName' columns from the DataFrame as input argument directly. In order to generate ID's for these two columns, 'CName' and 'BName' columns are first converted to lists and then RDD's are made of the lists with index zipped.
The function convert_id() is created to convert the list of names to lists of ID's.

In [21]:
list_b = df1.select('BName').rdd.map(lambda x: x).collect()

In [22]:
#list_b

In [23]:
list_c = df1.select('CName').rdd.flatMap(lambda x: x).collect()

In [24]:
#list_c

In above, flatMap() and map() both work, since there is only one element per line in the lists created.

In [25]:
def convert_id(list):
    row_id = 0
    Check = None
    id_list = []
    for name in list:
        if Check != name:
            row_id += 1
            Check = name
            id_list.append(row_id)
        else:
            row_id += 0   
            id_list.append(row_id)
    return id_list


Convert lists of names to lists of ID's:

In [26]:
list_bid = convert_id(list_b)

In [27]:
type(list_bid)

list

In [28]:
list_cid = convert_id(list_c)

In [29]:
#type(list_cid)

Generate an index for each ID:

RDD and DF with index for BID (ID for BName):

In [30]:
rdd_bid = sc.parallelize(list_bid).zipWithIndex()

In [31]:
rdd_bid.take(3)

[(1, 0), (1, 1), (1, 2)]

In [32]:
rdd_bid_col = rdd_bid.map(lambda r: Row(BID = r[0], BID_index = r[1]+1))

In [33]:
rdd_bid_col.take(3)

[Row(BID=1, BID_index=1), Row(BID=1, BID_index=2), Row(BID=1, BID_index=3)]

In [34]:
df_bid = rdd_bid_col.toDF()

In [35]:
df_bid.take(3)

[Row(BID=1, BID_index=1), Row(BID=1, BID_index=2), Row(BID=1, BID_index=3)]

RDD and DF with index for CID (ID for CName):

In [36]:
rdd_cid = sc.parallelize(list_cid).zipWithIndex()

In [37]:
rdd_cid.take(3)

[(1, 0), (1, 1), (1, 2)]

In [38]:
rdd_cid_col = rdd_cid.map(lambda r: Row(CID = r[0], CID_index = r[1]+1))

In [39]:
rdd_cid_col.take(3)

[Row(CID=1, CID_index=1), Row(CID=1, CID_index=2), Row(CID=1, CID_index=3)]

In [40]:
df_cid = rdd_cid_col.toDF()

In [41]:
df_cid.take(3)

[Row(CID=1, CID_index=1), Row(CID=1, CID_index=2), Row(CID=1, CID_index=3)]

#### Join the DataFrames to add the BID and CID columns 

In [42]:
df2 = df1.join(df_bid, df1.VID == df_bid.BID_index, 'left').drop('BID_index')

In [43]:
df2.take(1)

[Row(VID=26, VName='Gen.1:26', verse='And God said, Let us make man in our image, after our likeness: and let them have dominion over the fish of the sea, and over the fowl of the air, and over the cattle, and over all the earth, and over every creeping thing that creepeth upon the earth.', BName='Gen', CName='Gen.1', BID=1)]

In [44]:
df3 = df2.join(df_cid, df2.VID == df_cid.CID_index, 'left').drop('CID_index')

In [45]:
df3.take(1)

[Row(VID=26, VName='Gen.1:26', verse='And God said, Let us make man in our image, after our likeness: and let them have dominion over the fish of the sea, and over the fowl of the air, and over the cattle, and over all the earth, and over every creeping thing that creepeth upon the earth.', BName='Gen', CName='Gen.1', BID=1, CID=1)]

In [46]:
df3.show(2)

+---+--------+--------------------+-----+-----+---+---+
|VID|   VName|               verse|BName|CName|BID|CID|
+---+--------+--------------------+-----+-----+---+---+
| 26|Gen.1:26|And God said, Let...|  Gen|Gen.1|  1|  1|
| 29|Gen.1:29|And God said, Beh...|  Gen|Gen.1|  1|  1|
+---+--------+--------------------+-----+-----+---+---+
only showing top 2 rows



#### Start to perform Spark sql operations 

In [47]:
df3.createOrReplaceTempView('df3_table')

In [48]:
spark.sql("select BID, BName, CID, CName, VID, VName from df3_table where VName == '1John.2:3'").show()

+---+-----+----+-------+-----+---------+
|BID|BName| CID|  CName|  VID|    VName|
+---+-----+----+-------+-----+---------+
| 62|1John|1161|1John.2|30554|1John.2:3|
+---+-----+----+-------+-----+---------+



In [49]:
spark.sql("select distinct BID, CID from df3_table where CName == 'Luke.2'").show()

+---+---+
|BID|CID|
+---+---+
| 42|975|
+---+---+



In [50]:
spark.sql("select distinct BID, BName from df3_table order by BID desc").take(3)

[Row(BID=66, BName='Rev'),
 Row(BID=65, BName='Jude'),
 Row(BID=64, BName='3John')]

Find total number of verses in the book of John:

In [51]:
spark.sql("select count(BName) from df3_table where BName == 'John' ").show()

+------------+
|count(BName)|
+------------+
|         879|
+------------+



In [52]:
spark.sql("select count(VName) from df3_table where BName == 'John' ").show()

+------------+
|count(VName)|
+------------+
|         879|
+------------+



Find total number of chapters in the book of John:

In [53]:
spark.sql("select count(distinct CName) from df3_table where BName == 'John' ").show()

+---------------------+
|count(DISTINCT CName)|
+---------------------+
|                   21|
+---------------------+



Find total number of verses in the Bible:

In [60]:
spark.sql("select count(VName) from df3_table").show()

+------------+
|count(VName)|
+------------+
|       31102|
+------------+



Find total number of chapters in the Bible:

In [54]:
spark.sql("select count(distinct CName) from df3_table ").show()

+---------------------+
|count(DISTINCT CName)|
+---------------------+
|                 1189|
+---------------------+



Total number of books in the Bible:

In [55]:
spark.sql("select count(distinct BName) from df3_table ").show()

+---------------------+
|count(DISTINCT BName)|
+---------------------+
|                   66|
+---------------------+



The max number of characters that a verse can have in the Bible:

In [58]:
spark.sql("select max(length(verse)) from df3_table").show()

+------------------+
|max(length(verse))|
+------------------+
|               528|
+------------------+



The top longest verses:

In [81]:
spark.sql("select VID, VName, length(verse) from df3_table order by length(verse) desc").show(3)

+-----+----------+-------------+
|  VID|     VName|length(verse)|
+-----+----------+-------------+
|12827|  Esth.8:9|          528|
| 9979|2Kgs.16:15|          443|
|19448|  Jer.21:7|          442|
+-----+----------+-------------+
only showing top 3 rows



The shortest verses:

In [80]:
spark.sql("select VID, VName, length(verse) from df3_table order by length(verse) asc").show(3)

+-----+-----------+-------------+
|  VID|      VName|length(verse)|
+-----+-----------+-------------+
|26559| John.11:35|           11|
|29638|1Thess.5:16|           17|
|10278|  1Chr.1:25|           17|
+-----+-----------+-------------+
only showing top 3 rows



The shortest chapters in terms of total number of characters: 

In [112]:
spark.sql("select CName, sum(length(verse)) from df3_table group by CName order by sum(length(verse)) asc").show(5)

+------+------------------+
| CName|sum(length(verse))|
+------+------------------+
|Ps.117|               173|
|Ps.134|               221|
|Ps.131|               302|
|Ps.133|               369|
|Job.25|               428|
+------+------------------+
only showing top 5 rows



The longest chapters in terms of total number of characters: 

In [114]:
spark.sql("select CName, sum(length(verse)) from df3_table group by CName order by sum(length(verse)) desc").show(5)

+-------+------------------+
|  CName|sum(length(verse))|
+-------+------------------+
| Ps.119|             12839|
| 1Kgs.8|             11318|
|Deut.28|             10807|
|  Num.7|             10395|
|Ezek.16|              9888|
+-------+------------------+
only showing top 5 rows



The shortest chapters in terms of total number of verses: 

In [104]:
spark.sql("select CName, count(CName) from df3_table group by CName order by count(CName) asc").show(5)

+-------+------------+
|  CName|count(CName)|
+-------+------------+
| Ps.117|           2|
| Ps.134|           3|
| Ps.133|           3|
| Ps.131|           3|
|Esth.10|           3|
+-------+------------+
only showing top 5 rows



The longest chapters in terms of total number of verses: 

In [105]:
spark.sql("select CName, count(CName) from df3_table group by CName order by count(CName) desc").show(5)

+-------+------------+
|  CName|count(CName)|
+-------+------------+
| Ps.119|         176|
|  Num.7|          89|
| 1Chr.6|          81|
| Luke.1|          80|
|Matt.26|          75|
+-------+------------+
only showing top 5 rows



The shortest books in terms of total number of verses: 

In [106]:
spark.sql("select BName, count(BName) from df3_table group by BName order by count(BName) asc").show(5)

+-----+------------+
|BName|count(BName)|
+-----+------------+
|2John|          13|
|3John|          14|
| Obad|          21|
| Jude|          25|
| Phlm|          25|
+-----+------------+
only showing top 5 rows



The longest books in terms of total number of verses: 

In [107]:
spark.sql("select BName, count(BName) from df3_table group by BName order by count(BName) desc").show(5)

+-----+------------+
|BName|count(BName)|
+-----+------------+
|   Ps|        2461|
|  Gen|        1533|
|  Jer|        1364|
|  Isa|        1292|
|  Num|        1288|
+-----+------------+
only showing top 5 rows



The longest books in terms of total number of characters:

In [126]:
spark.sql("select BName, sum(length(verse)) from df3_table group by BName order by sum(length(verse)) desc").show(5)

+-----+------------------+
|BName|sum(length(verse))|
+-----+------------------+
|  Jer|            222753|
|   Ps|            221635|
| Ezek|            204283|
|  Gen|            195318|
|  Isa|            192879|
+-----+------------------+
only showing top 5 rows



The shortest books in terms of total number of characters:

In [127]:
spark.sql("select BName, sum(length(verse)) from df3_table group by BName order by sum(length(verse)) asc").show(5)

+-----+------------------+
|BName|sum(length(verse))|
+-----+------------------+
|2John|              1540|
|3John|              1579|
| Phlm|              2295|
| Jude|              3498|
| Obad|              3561|
+-----+------------------+
only showing top 5 rows



#### Query with Spark DataFrame functions:

In [738]:
df3.filter(df3.verse.contains('Lords')).collect()

[Row(VID=31034, VName='Rev.19:16', verse='And he hath on his vesture and on his thigh a name written, King Of Kings, And Lord Of Lords .', BName='Rev', CName='Rev.19', BID=66, CID=1186)]

In [739]:
df3.filter(df3.verse.endswith('Lords .')).collect()

[Row(VID=31034, VName='Rev.19:16', verse='And he hath on his vesture and on his thigh a name written, King Of Kings, And Lord Of Lords .', BName='Rev', CName='Rev.19', BID=66, CID=1186)]

In [740]:
df3.filter(df3.verse.like('%ords .')).collect()

[Row(VID=31034, VName='Rev.19:16', verse='And he hath on his vesture and on his thigh a name written, King Of Kings, And Lord Of Lords .', BName='Rev', CName='Rev.19', BID=66, CID=1186)]

In [741]:
#df3[df3.CName.isin("1John.1", "Ezek.59")].collect()

In [742]:
#df3.filter(df3.CName.like('%zek.39%')).collect()

In [743]:
#df3.groupBy('BName').count().show()

In [757]:
#df3.orderBy(desc('BID'), 'VID').show()

In [95]:
#df3.first()

In [96]:
#df3.head()

In [97]:
#df3.limit(1).collect()

In [87]:
df3.count()

31102

In [91]:
df3.select('CID', 'VID', 'VName').limit(3).collect()

[Row(CID=1, VID=26, VName='Gen.1:26'),
 Row(CID=1, VID=29, VName='Gen.1:29'),
 Row(CID=19, VID=474, VName='Gen.19:16')]

In [94]:
df3.sort(df3.VID.asc()).limit(3).collect()

[Row(VID=1, VName='Gen.1:1', verse='In the beginning God created the heaven and the earth.', BName='Gen', CName='Gen.1', BID=1, CID=1),
 Row(VID=2, VName='Gen.1:2', verse='And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.', BName='Gen', CName='Gen.1', BID=1, CID=1),
 Row(VID=3, VName='Gen.1:3', verse='And God said, Let there be light: and there was light.', BName='Gen', CName='Gen.1', BID=1, CID=1)]